In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime
import os

C:\Users\denni\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\denni\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\denni\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os
import datetime

# Load dataset
img_size = (64, 64)
batch_size = 32
data_dir = "C:/Users/denni/cogitron/dataset"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

# Normalize pixel values
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Improve performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# TensorBoard callback
log_dir = os.path.join("logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Define CNN model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')  # 3 classes: green_light, red_light, no_light
])


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



# Train model with TensorBoard logging
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tensorboard_callback]
)

# Save model
model.save("light_classifier.h5")


Found 1079 files belonging to 3 classes.
Using 864 files for training.
Found 1079 files belonging to 3 classes.
Using 215 files for validation.
Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.7083 - loss: 0.7168 - val_accuracy: 0.9256 - val_loss: 0.2582
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.9387 - loss: 0.1848 - val_accuracy: 0.9767 - val_loss: 0.1021
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.9769 - loss: 0.0763 - val_accuracy: 0.9953 - val_loss: 0.0277
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.9919 - loss: 0.0296 - val_accuracy: 0.9953 - val_loss: 0.0267
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 1.0000 - loss: 0.0085 - val_accuracy: 0.9953 - val_loss: 0.0143
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 1.0000 - loss: 0.0038 - val_accuracy: 0.9953 - val_loss: 0.0082
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 1.0000 - loss: 0.0027 - val_acc

In [9]:
%reload_ext tensorboard
%tensorboard --logdir=fit==logs/fit


In [ ]:
import os
print(os.listdir("logs/fit"))



['20250930-195503']


In [11]:
import os
print(os.listdir("C:/Users/denni/cogitron/computer-vision/logs/fit"))


['20251007-193809', '20251007-193927']


In [12]:
print(train_ds.class_names)

AttributeError: '_PrefetchDataset' object has no attribute 'class_names'

In [ ]:
%tensorboard --logdir=fit==logs/fit


Reusing TensorBoard on port 6006 (pid 15716), started 0:06:11 ago. (Use '!kill 15716' to kill it.)

In [ ]:
import os

log_subdirs = os.listdir(log_dir)
print("Log subdirs:", log_subdirs)

for subdir in log_subdirs:
    files = os.listdir(os.path.join(log_dir, subdir))
    print(f"Files in {subdir}: {files}")


Log subdirs: ['train', 'validation']
Files in train: ['events.out.tfevents.1759254575.ODIN-LAPTOP.20788.8.v2']
Files in validation: ['events.out.tfevents.1759254576.ODIN-LAPTOP.20788.9.v2']


In [ ]:
import tensorflow as tf
import datetime
import os

log_dir = "logs/test/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Dummy dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[..., None]/255.0
x_test = x_test[..., None]/255.0

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28,28,1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 43s 4us/step


C:\Users\ovank\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8782 - loss: 0.4669 - val_accuracy: 0.9139 - val_loss: 0.3085


In [13]:
print(os.listdir("C:/Users/denni/cogitron/dataset"))

['green_light', 'no_light', 'red_light']
